In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype = dtype_dict)
training = pd.read_csv('kc_house_train_data.csv', dtype = dtype_dict)
testing = pd.read_csv('kc_house_test_data.csv', dtype = dtype_dict)

sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


Next, from Module 2 (Multiple Regression), copy and paste the ‘get_numpy_data’ function (or equivalent) that takes a data set, a list of features (e.g. [‘sqft_living’, ‘bedrooms’]), to be used as inputs, and a name of the output (e.g. ‘price’). This function returns a ‘feature_matrix’ (2D array) consisting of first a column of ones followed by columns containing the values of the input features in the data set in the same order as the input list. It also returns an ‘output_array’ which is an array of the values of the output in the data set (e.g. ‘price’).

In [3]:
def get_numpy_data(data_frame, features, output):
    data_frame['constant'] = 1 # add a constant column to an DataFrame
    # prepend variable 'constant' to the features list
    features = ['constant'] + features

    features_dataframe = data_frame[features]

    features_matrix = features_dataframe.as_matrix()
 
    output_dataframe = data_frame[output]
    output_array = output_dataframe.as_matrix()

    return(features_matrix, output_array)

Similarly, copy and paste the ‘predict_output’ function (or equivalent) from Module 2. This function accepts a 2D array ‘feature_matrix’ and a 1D array ‘weights’ and return a 1D array ‘predictions’.

In [4]:
def predict_outcome(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

In the house dataset, features vary wildly in their relative magnitude: ‘sqft_living’ is very large overall compared to ‘bedrooms’, for instance. As a result, weight for ‘sqft_living’ would be much smaller than weight for ‘bedrooms’. This is problematic because “small” weights are dropped first as l1_penalty goes up.

To give equal considerations for all features, we need to normalize features as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Write a short function called ‘normalize_features(feature_matrix)’, which normalizes columns of a given feature matrix. The function should return a pair ‘(normalized_features, norms)’, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data.

In [5]:
def normalize_features(features):
    norms = np.linalg.norm(features, axis = 0)
    normalized_features = features / norms
    return(normalized_features, norms)

### Review of Coordinate Descent

We seek to obtain a sparse set of weights by minimizing the LASSO cost function

SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).

(By convention, we do not include w[0] in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use coordinate descent: at each iteration, we will fix all weights but weight i and find the value of weight i that minimizes the objective. That is, we look for

argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]

where all weights other than w[i] are held to be constant. We will optimize one w[i] at a time, circling through the weights multiple times.

 *   Pick a coordinate i
 *   Compute w[i] that minimizes the LASSO cost function
 *   Repeat the two steps for all coordinates, multiple times

For this assignment, we use cyclical coordinate descent with normalized features, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. 

### Effect of L1 penalty

Consider a simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. The output is ‘price’.

 *   First, run get_numpy_data() (or equivalent) to obtain a feature matrix with 3 columns (constant column added). Use the entire ‘sales’ dataset for now.
 *   Normalize columns of the feature matrix. Save the norms of original features as ‘norms’.
 *   Set initial weights to [1,4,1].
 *   Make predictions with feature matrix and initial weights.
 *   Compute values of ro[i]

In [6]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

normalize columns of the feature matrix

In [7]:
(normalized_simple_feature_matrix, norms) = normalize_features(simple_feature_matrix)

Make predictions with feature matrix and initial weights

In [8]:
initial_weights = np.array([1,4,1])
prediction = predict_outcome(normalized_simple_feature_matrix, initial_weights)

Compute values of ro[i]

ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]

In [9]:
ro = [0 for i in range(normalized_simple_feature_matrix.shape[1])]

for j in range(normalized_simple_feature_matrix.shape[1]):
    ro[j] = (normalized_simple_feature_matrix[:, j] * (output - prediction + initial_weights[j] * normalized_simple_feature_matrix[:,j])).sum()
    
ro

[79400300.014522895, 87939470.823251754, 80966698.66623947]

# Question 1
From the section "Effect of L1 penalty": Consider the simple model with 2 features trained on the entire sales dataset.

Which of the following values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a coordinate gradient step in that coordinate? (Select all that apply)



In [10]:
np.array(ro) * 2

array([  1.58800600e+08,   1.75878942e+08,   1.61933397e+08])

# Question 2
Refer to the same model as the previous question.

Which of the following values of l1_penalty would set both w[1] and w[2] to zero, if we were to take a coordinate gradient step in that coordinate? (Select all that apply)

In [11]:
np.array(ro) * 2

array([  1.58800600e+08,   1.75878942e+08,   1.61933397e+08])

### Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [12]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_outcome(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i] * (output - prediction + weights[i] * feature_matrix[:,i])).sum()
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty / 2.0
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty / 2.0
    else:
        new_weight_i = 0.
    
    return new_weight_i

If you are using Numpy, test your function with the following snippet:

In [13]:
# should print 0.425558846691
import math
lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.42555884669102512

### Cyclical coordinate descent

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:

 *   As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
 *   After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to the previous step.

### Return weights

The function should accept the following parameters:

 *   Feature matrix
 *   Output array
 *   Initial weights
 *   L1 penalty
 *   Tolerance

In [14]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    D = feature_matrix.shape[1]
    weights = np.array(initial_weights)
    weight_change = np.array(initial_weights) * 0.0
    converged = False
    
    while not converged:
        for index in range(D):
            new_weight = lasso_coordinate_descent_step(index, feature_matrix, output, weights, l1_penalty)
            
            weight_change[index] = np.abs(new_weight - weights[index])
            
            weights[index] = new_weight
            
        if max(weight_change) < tolerance:
            converged = True
    
    return(weights)

Let us now go back to the simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. Using ‘get_numpy_data’ (or equivalent), extract the feature matrix and the output array from from the house dataframe. Then normalize the feature matrix using ‘normalized_features()’ function.

Using the following parameters, learn the weights on the sales dataset.

  *  Initial weights = all zeros
  *  L1 penalty = 1e7
  *  Tolerance = 1.0

In [15]:
initial_weights = np.zeros(simple_feature_matrix.shape[1])
l1_penalty = 1e7
tolerance = 1.0

weights_q3 = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output, initial_weights, l1_penalty, tolerance)

# Question 3

From the section "Cyclical coordinate descent": Using the simple model (with 2 features), we run our implementation of LASSO coordinate descent on the normalized sales dataset. We apply an L1 penalty of 1e7 and tolerance of 1.0.

Which of the following ranges contains the RSS of the learned model on the normalized dataset?

In [16]:
def RSS_calculation(feature_matrix, weights, output):
    prediction = predict_outcome(feature_matrix, weights)
    residual = output - prediction
    RSS = (residual ** 2).sum()
    return RSS 

In [17]:
print(RSS_calculation(normalized_simple_feature_matrix, weights_q3, output))

1.63049247672e+15


# Question 4
Refer to the same model as the previous question.

Which of the following features were assigned a zero weight at convergence?

In [18]:
weights_q3

array([ 21624997.95951909,  63157247.20788956,         0.        ])

### Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets. If you are using GraphLab Create, call ‘random_split’ with .8 ratio and seed=0. Otherwise, please down the corresponding csv files from the downloads section.

Create a normalized feature matrix from the TRAINING data with the following set of features.

 *   bedrooms, bathrooms, sqft_living, sqft_lot, floors, waterfront, view, condition, grade, sqft_above, sqft_basement, yr_built, yr_renovated

Make sure you store the norms for the normalization, since we’ll use them later.

First, learn the weights with l1_penalty=1e7, on the training data. Initialize weights to all zeros, and set the tolerance=1. Call resulting weights’ weights1e7’, you will need them later.

In [19]:
more_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

(more_feature_matrix, more_feature_output) = get_numpy_data(training, more_features, my_output)

(normalized_more_feature_matrix, more_feature_norms) = normalize_features(more_feature_matrix)

initial_weights = np.zeros(normalized_more_feature_matrix.shape[1])
l1_penalty = 1e7
tolerance = 1.0

weights1e7 = lasso_cyclical_coordinate_descent(normalized_more_feature_matrix, more_feature_output, initial_weights, l1_penalty, tolerance)

# Question 5
In the section "Evaluating LASSO fit with more features", we split the data into training and test sets and learn weights with varying degree of L1 penalties. The model now has 13 features.

In the model trained with l1_penalty=1e7, which of the following features has non-zero weight? (Select all that apply)

In [20]:
nonzero_weight_index = [np.where(weights1e7 == weight)[0][0] for weight in weights1e7 if weight !=0]
feature_list = ['constant'] + more_features
nonzero_weight_features = [feature_list[index] for index in nonzero_weight_index]
nonzero_weight_features

['constant', 'sqft_living', 'waterfront', 'view']

In [21]:
def return_nonzero_weight_features(weights):
    nonzero_weight_index = [np.where(weights == weight)[0][0] for weight in weights if weight !=0]
    feature_list = ['constant'] + more_features
    nonzero_weight_features = [feature_list[index] for index in nonzero_weight_index]
    return nonzero_weight_features

return_nonzero_weight_features(weights1e7)

['constant', 'sqft_living', 'waterfront', 'view']

# Question 6

This question refers to the same model as the previous question.

In the model trained with l1_penalty=1e8, which of the following features has non-zero weight? (Select all that apply)

In [22]:
weights1e8 = lasso_cyclical_coordinate_descent(normalized_more_feature_matrix, more_feature_output, initial_weights, 1e8, tolerance)

return_nonzero_weight_features(weights1e8)

['constant']

# Question 7
This question refers to the same model as the previous question.

In the model trained with l1_penalty=1e4, which of the following features has non-zero weight? (Select all that apply)

In [23]:
weights1e4 = lasso_cyclical_coordinate_descent(normalized_more_feature_matrix, more_feature_output, initial_weights, 1e4, tolerance)

return_nonzero_weight_features(weights1e4)

['constant',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

# Question 8
In the section "Evaluating each of the learned models on the test data", we evaluate three models on the test data. The three models were trained with same set of features but different L1 penalties.

Which of the three models gives the lowest RSS on the TEST data?

In [26]:
(more_feature_test_matrix, more_feature_test_output) = get_numpy_data(testing, more_features, my_output)
(normalized_more_feature_test_matrix, more_feature_test_norms) = normalize_features(more_feature_test_matrix)

RSS_1e7 = RSS_calculation(more_feature_test_matrix, weights1e7, more_feature_test_output)
RSS_1e8 = RSS_calculation(more_feature_test_matrix, weights1e8, more_feature_test_output)
RSS_1e4 = RSS_calculation(more_feature_test_matrix, weights1e4, more_feature_test_output)
print("l1 penalty 1e4 RSS: ", RSS_1e4)
print("l1 penalty 1e7 RSS: ", RSS_1e7)
print("l1 penalty 1e8 RSS: ", RSS_1e8)

l1 penalty 1e4 RSS:  1.19751759598e+28
l1 penalty 1e7 RSS:  5.09512950079e+25
l1 penalty 1e8 RSS:  2.10624234224e+19


### Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data. Extract the feature matrix and output array from the TEST set. But this time, do NOT normalize the feature matrix. Instead, use the normalized version of weights to make predictions.

Compute the RSS of each of the three normalized weights on the (unnormalized) feature matrix.

In [28]:
normalized_weights1e4 = weights1e4 / more_feature_norms
normalized_weights1e7 = weights1e7 / more_feature_norms
normalized_weights1e8 = weights1e8 / more_feature_norms

RSS_1e7_normalized = RSS_calculation(more_feature_test_matrix, normalized_weights1e7, more_feature_test_output)
RSS_1e8_normalized = RSS_calculation(more_feature_test_matrix, normalized_weights1e8, more_feature_test_output)
RSS_1e4_normalized = RSS_calculation(more_feature_test_matrix, normalized_weights1e4, more_feature_test_output)

print("l1 penalty 1e4 RSS: ", RSS_1e4_normalized)
print("l1 penalty 1e7 RSS: ", RSS_1e7_normalized)
print("l1 penalty 1e8 RSS: ", RSS_1e8_normalized)

l1 penalty 1e4 RSS:  1.94415789314e+14
l1 penalty 1e7 RSS:  2.7596207592e+14
l1 penalty 1e8 RSS:  5.37166151497e+14
